In [259]:
import cv2
import numpy as np
from scipy import stats
from itertools import product

In [16]:
def normalizeImage(img):
    maxValue=np.amax(img)
    return img/maxValue,int(maxValue)

def removeFloor(img_array,threshold1):
    height = img_array.shape[0]
    width = img_array.shape[1]
    cy=int(height/2)  #center y
    fy=580  #focal length y
    z=5000  #initial min depth

    croppedRow = img_array[-1][np.nonzero(img_array[-1])]
    #z=stats.mode(img_array[-1]).mode[0]
    if (len(croppedRow)>0):
        z=stats.mode(croppedRow).mode[0]     #initial floor depth
    else:
        z=75

    Y=z*(height-cy)/fy                       #actual y of floor
    Y=Y-threshold1
    
    for i in range(height-1,cy-1,-1):
        z1=(Y*fy)/((i+1)-cy)                   #depth of Y height

        for j in range(width):
            if (img_array[i][j]>z1):
                img_array[i][j]=0

    output,__=normalizeImage(img_array)
    return output

def removeFar(img,threshold):
    maxVal = np.amax(img)
    ret,output = cv2.threshold(img,threshold,maxVal,cv2.THRESH_TOZERO_INV)
    return output

In [110]:
class UFarray:
    def __init__(self):
        # Array which holds label -> set equivalences
        self.P = []

        # Name of the next label, when one is created
        self.label = 0

    def makeLabel(self):
        r = self.label
        self.label += 1
        self.P.append(r)
        return r
    
    # Makes all nodes "in the path of node i" point to root
    def setRoot(self, i, root):
        while self.P[i] < i:
            j = self.P[i]
            self.P[i] = root
            i = j
        self.P[i] = root

    # Finds the root node of the tree containing node i
    def findRoot(self, i):
        while self.P[i] < i:
            i = self.P[i]
        return i
    
    # Finds the root of the tree containing node i
    # Simultaneously compresses the tree
    def find(self, i):
        root = self.findRoot(i)
        self.setRoot(i, root)
        return root
    
    # Joins the two trees containing nodes i and j
    # Modified to be less agressive about compressing paths
    # because performance was suffering some from over-compression
    def union(self, i, j):
        if i != j:
            root = self.findRoot(i)
            rootj = self.findRoot(j)
            if root > rootj: root = rootj
            self.setRoot(j, root)
            self.setRoot(i, root)
    
    def flatten(self):
        for i in range(1, len(self.P)):
            self.P[i] = self.P[self.P[i]]


def get_components(frame, threshold):
    width, height = frame.shape[1], frame.shape[0]
 
    # Union find data structure
    uf = UFarray()
 
    # Dictionary of point:label pairs
    labels = {}
 
    for y, x in product(range(height), range(width)):

 
        #
        # Pixel names were chosen as shown:
        #
        #   -------------
        #   | a | b | c |
        #   -------------
        #   | d | e |   |
        #   -------------
        #   |   |   |   |
        #   -------------
        #
        # The current pixel is e
        # a, b, c, and d are its neighbors of interest
        #
        # 255 is white, 0 is black
        # White pixels part of the background, so they are ignored
        # If a pixel lies outside the bounds of the image, it default to white
        #
 
        # If the current pixel is white, it's obviously not a component...
        if frame[y, x] == 0:
            pass
 
        # If pixel b is in the image and black:
        #   a, d, and c are its neighbors, so they are all part of the same component
        #   Therefore, there is no reason to check their labels
        #   so simply assign b's label to e
        elif y > 0 and abs(frame[y-1,x]-frame[y,x]) < threshold:
            labels[y,x] = labels[(y-1,x)]
 
        # If pixel c is in the image and black:
        #   b is its neighbor, but a and d are not
        #   Therefore, we must check a and d's labels
        elif x+1 < width and y > 0 and abs(frame[y-1,x+1]-frame[y,x]) < threshold:
            c = labels[(y-1),x+1]
            labels[y,x] = c
 
            # If pixel a is in the image and black:
            #   Then a and c are connected through e
            #   Therefore, we must union their sets
            if x > 0 and abs(frame[y-1,x-1]-frame[y, x]) < threshold:
                a = labels[(y-1,x-1)]
                uf.union(c, a)
 
            # If pixel d is in the image and black:
            #   Then d and c are connected through e
            #   Therefore we must union their sets
            elif x > 0 and abs(frame[y, x-1]-frame[y, x]) < threshold:
                d = labels[(y, x-1)]
                uf.union(c, d)
 
        # If pixel a is in the image and black:
        #   We already know b and c are white
        #   d is a's neighbor, so they already have the same label
        #   So simply assign a's label to e
        elif x > 0 and y > 0 and abs(frame[y-1,x-1]-frame[y,x]) < threshold:
            labels[y,x] = labels[(y-1,x-1)]
 
        # If pixel d is in the image and black
        #   We already know a, b, and c are white
        #   so simpy assign d's label to e
        elif x > 0 and abs(frame[y, x-1]-frame[y, x]) < threshold:
            labels[y, x] = labels[(y, x-1)]
 
        # All the neighboring pixels are white,
        # Therefore the current pixel is a new component
        else: 
            labels[y, x] = uf.makeLabel()
  
    uf.flatten()

    for (y, x) in labels:
        # Name of the component the current point belongs to
        component = uf.find(labels[(y, x)])
        # Update the labels with correct information
        labels[(y, x)] = component
 
    return labels

def getStatsOfLabel(frame,label,labels):
    img = np.zeros(frame.shape)
    for y, x in product(range(img.shape[0]), range(img.shape[1])):
        if (y,x) in labels and labels[(y,x)] == label:
            img[y,x]=255
        else:
            img[y,x]=0
    
    img = img.astype(np.uint8)
    contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_TC89_L1)
    x,y,w,h = cv2.boundingRect(contours[0])
    return [x+w/2,y+h/2],w,h

def getCentroid(img):
    img,__=normalizeImage(img)
    img = (img*255).astype(np.uint8)
    cnts,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_TC89_L1)
    boxes = []
    for c in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        boxes.append([x,y, x+w,y+h])

    boxes = np.asarray(boxes)
    # need an extra "min/max" for contours outside the frame
    left = np.min(boxes[:,0])
    top = np.min(boxes[:,1])
    right = np.max(boxes[:,2])
    bottom = np.max(boxes[:,3])

    return [(left+right)/2,(top+bottom)/2]

In [287]:
depthFrames = np.fromfile(<envFilePath>, dtype = "uint16")
numOfFrames = int(depthFrames.shape[0]/(640*480))
depthFrames = depthFrames.reshape((numOfFrames, 480, 640))
numOfFrames = numOfFrames/2

envDepthFrames = depthFrames[::2]
envFrameCount = 30 if numOfFrames>30 else numOfFrames
smoothedEnv = np.zeros(envDepthFrames[0].shape)
q1 = int(envFrameCount/4)

for frameNum in range(q1,q1+10):
    img=envDepthFrames[frameNum].astype(np.float32)
    img,maxDepth=normalizeImage(img)
    #cv2.imshow('env',img)
    floorRemoved,__=normalizeImage(removeFloor(img*255,9))
    #cv2.imshow('floorRemoved',floorRemoved)
    floorRemoved = floorRemoved*maxDepth
    distantFiltered = removeFar(floorRemoved,4000)
    #cv2.imshow('distantFiltered',normalizeImage(distantFiltered)[0])
    smoothedEnv = smoothedEnv + (distantFiltered/(10))
    #cv2.waitKey()
    
cv2.imshow('smoothedEnv',normalizeImage(smoothedEnv)[0])
cv2.waitKey()
cv2.destroyAllWindows()

In [288]:
depthFilePath = <depthFilePath>
depthFrames = np.fromfile(depthFilePath, dtype = "uint16")
numOfFrames = int(depthFrames.shape[0]/(640*480))
depthFrames = depthFrames.reshape((numOfFrames, 480, 640))
numOfFrames = numOfFrames/2

env = smoothedEnv
silhoutte=[]
actionFrameCount = int(numOfFrames)
previousSilhouttePosition=None
lastFrame = None

for frameNum in range(actionFrameCount):
    silhouetteAvailable = True

    original = depthFrames[::2][frameNum].astype(np.float64)

    normalizedOriginal,maxDepth = normalizeImage(original)
    cv2.imshow('original',normalizedOriginal)
    
    #remove floor
    floorRemoved=removeFloor(normalizedOriginal*255,9)        
    floorRemoved,__=normalizeImage(floorRemoved)
    floorRemoved=floorRemoved*maxDepth
    
    #remove background beyond threshold
    distantFiltered = removeFar(floorRemoved,4000)
    foreground = distantFiltered-env
    img = foreground.astype(np.float32)  
    ret,binaryImg = cv2.threshold(normalizeImage(img)[0],0.6,1,cv2.THRESH_BINARY)
    realDepthSilhouette = (binaryImg*normalizedOriginal)
    #ccl
    labels = get_components(realDepthSilhouette,0.02)
    Lbls, lbl_count = np.unique(np.array([i for i in labels.values()]),return_counts=True)
    
    croppedLabels = []
    croppedLblsWithCentroids={} 

    for i in range(len(Lbls)):
        if(lbl_count[i]>375 and lbl_count[i]<55000):
            croppedLabels.append(Lbls[i])
            centroids,width,height=getStatsOfLabel(realDepthSilhouette,Lbls[i],labels)
            if(lbl_count[i] not in croppedLblsWithCentroids):
                newLblCount=lbl_count[i]+i*0.001   ###to avoid replace of duplicate label counts
                croppedLblsWithCentroids[newLblCount]=centroids,(width,height),Lbls[i]

    if(len(croppedLblsWithCentroids)>1):

        while True: 
            if(len(croppedLblsWithCentroids)>0):
                centroidOfMax = croppedLblsWithCentroids[max(croppedLblsWithCentroids)][0]
                if(previousSilhouttePosition == None):
                    break
                elif(abs(centroidOfMax[0]-previousSilhouttePosition[0])>200 or abs(centroidOfMax[1]-previousSilhouttePosition[1])>200):
                    croppedLabels.remove(croppedLblsWithCentroids[max(croppedLblsWithCentroids)][2])
                    croppedLblsWithCentroids.pop(max(croppedLblsWithCentroids))
                else:
                    break
            else:
                silhouetteAvailable = False
                break
    

    if (silhouetteAvailable and len(croppedLblsWithCentroids)>0):
        centroidOfMax = croppedLblsWithCentroids[max(croppedLblsWithCentroids)][0]
        sizeOfMax = croppedLblsWithCentroids[max(croppedLblsWithCentroids)][1]
        for croppedLabel in croppedLblsWithCentroids:
            labelX = croppedLblsWithCentroids[croppedLabel][0][0]
            labelY = croppedLblsWithCentroids[croppedLabel][0][1]
            labelName = croppedLblsWithCentroids[croppedLabel][2]

            if(abs(labelX-centroidOfMax[0])>1.2*sizeOfMax[0] or abs(labelY-centroidOfMax[1])>1.5*sizeOfMax[1]):
                croppedLabels.remove(labelName)
    
    if (len(croppedLblsWithCentroids)==0):
        silhouetteAvailable = False

    if (silhouetteAvailable):
        img = np.zeros(original.shape)

        for y, x in product(range(img.shape[0]), range(img.shape[1])):
            if (y,x) in labels and labels[(y,x)] in croppedLabels:
                img[y,x]=1
            else:
                img[y,x]=0

        cclImg =(original*img).astype(np.uint16)

        previousSilhouttePosition = getCentroid(cclImg)
        silhoutte.append(np.array(cclImg))
        lastFrame = frameNum
    else:
        if(len(silhoutte)>0):
            silhoutte.append(silhoutte[-1])
    cv2.imshow('Silhouette',normalizedOriginal*img)
    cv2.waitKey(1)
    
cv2.destroyAllWindows()    
    